In [ ]:
!pip install tf2onnx onnxruntime tqdm transformers -q

In [ ]:
import tensorflow as tf
import logging
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Conv1D,
    Dropout,
    Input,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from transformers import TFBertModel
model_name = "aubmindlab/bert-base-arabertv2"
max_length = 512
batch_size = 128
num_class = 95

tf.compat.v1.logging.set_verbosity(2)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.1.109.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.1.109.66:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
def arabert_encode(data):
    tokens = tokenizer.batch_encode_plus(
        data, max_length=max_length, padding="max_length", truncation=True
    )
    return tf.constant(tokens["input_ids"])


In [ ]:
def arabert_model():
    bert_encoder = TFBertModel.from_pretrained(model_name, output_attentions=True)
    input_word_ids = Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    last_hidden_states = bert_encoder(input_word_ids)[0]
    clf_output = Flatten()(last_hidden_states)
    net = Dense(512, activation="relu")(clf_output)
    net = Dropout(0.3)(net)
    net = Dense(440, activation="relu")(net)
    net = Dropout(0.3)(net)
    output = Dense(num_class, activation="softplus")(net)
    model = Model(inputs=input_word_ids, outputs=output)
    return model

In [ ]:
from keras import backend as K

steps_per_exe =32

with strategy.scope():
  model = arabert_model()
  adam_optimizer = Adam(learning_rate=1e-5)
  model.compile(
        loss="categorical_crossentropy",
        optimizer=adam_optimizer,
        metrics=['acc'],
        steps_per_execution=steps_per_exe
    )
#model.summary()

Some layers from the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at aubmindlab/bert-base-arabertv2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
!gdown --id 1-TkUYU6NBVu1LzDHroujQOLexMMwwUEO && gdown --id 13NtD5HBlX7-v9Y43hHuCSguN7BJkU4pK

Downloading...
From: https://drive.google.com/uc?id=1-TkUYU6NBVu1LzDHroujQOLexMMwwUEO
To: /content/checkpoint.zip
100% 3.42G/3.42G [00:45<00:00, 74.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=13NtD5HBlX7-v9Y43hHuCSguN7BJkU4pK
To: /content/authors.csv
100% 7.15k/7.15k [00:00<00:00, 6.28MB/s]


In [ ]:
!unzip -q checkpoint.zip

replace content/drive/MyDrive/ML/ARNLP/local/checkpoint/keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.load_weights('/content/content/drive/MyDrive/ML/ARNLP/local/checkpoint', options=save_locally)

In [ ]:
df = pd.read_csv('authors.csv', index_col=0)

In [ ]:
text = 'الرأي فإنه متى ما اتبع الرأي جاءه رجل آخر أقوى في الرأي منه فاتبعه فكلما غلبه رجل اتبعه أرى أن هذا بعد لم يتم واعملوا من الآثار بما روي عن جابر رضي الله عنه أن النبي صلى الله عليه وسلم قال قد تركت فيكم ما لن تضلوا بعدي إذا اعتصمتم به كتاب الله وسنتي ولن يتفرقا حتى يردا على الحوض وروي عن عمرو بن شعيب عن أبيه عن جده خرج رسول الله صلى الله عليه وسلم يوما وهم يجادلون في القرآن فخرج ووجهه أحمر كالدم فقال يا قوم على هذا هلك من كان قبلكم جادلوا في القرآن وضربوا بعضه ببعض فما كان من حلال فاعملوا به وما كان من حرام فانتهوا عنه وما كان من متشابه فآمنوا به'
text_encoded = arabert_encode(text)
p = (
    tf.data.Dataset.from_tensor_slices((text_encoded))
    .batch(batch_size)
).cache()
y_pred = model.predict(p, verbose=2)

In [ ]:
author , book = df.iloc[[np.argmax(x) for x in y_pred][0]].tolist()
print('Author: ',author , '\nBook: ', book, '\nConfidence:', y_pred[0][[np.argmax(x) for x in y_pred][0]])

Author:  ابن أمير حاج 
Book:  التقرير والتحبير.txt 
Confidence: 0.8255385
